# 01 - Exploratory Analysis

###### Here, histograms, boxplots and density graphs are constructed, taking into account the sum of hours reported to visually analyze the asymmetry and dispersion of the data set in relation to the type of product and department. The results obtained here influence the analyses carried out in subsequent stages.
---

#### Loading Packages and Database
----

In [2]:
import plotly.io as pio
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# import fast_dummies
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
from scipy.stats import triang, gaussian_kde
import os
import sys
from pathlib import Path

In [3]:
# Adicionar o diretório 'modules_py' ao sys.path
modules_path = Path("modules_py").resolve()
sys.path.append(str(modules_path))

from data_cleaning import DataCleaning_MyData, DataCleaning_GroupData, DataCleaning_OffCod, DataCleaning_frequents, DataCleaning_dummies

try:
    # Executar a função principal de limpeza de dados
    my_data = DataCleaning_MyData()
    clean_media = DataCleaning_GroupData()
    mais_frequentes = DataCleaning_frequents()
    dummies = DataCleaning_dummies()
    offcod = DataCleaning_OffCod()
    
    print("Dados processados com sucesso!")
    
    # Aqui pode verificar o conteúdo de cada tabela gerada no código
    display(my_data)

# Mostrar o erro caso não execute
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados processados com sucesso!


,Fonte,OFFCod,OFFEscopo,SomaDeHorasApontadas,Andrea_DEPARTAMENTO.Descricao,QuantidadePedida,TipodeProduto,PRODUTOS.Codigo,Produto.Nome,ITENS.Codigo,...,ITENS.Classe,Poder Linear,Poder Z,Capacidade(TSS/D),Cap Normal Linear,Cap Normal Z,Evaporacao(T/H),Evap Normal Linear,Evap Normal Z,SomaDeHorasApontadasUnitario
0,PropostaAjustada,134,PACKAGE 65 T/H,504.00,MONTAGEM,1.0,Caldeira,12.0,CC,1100.0,...,Operação/Projeto,12.000000,-100.000000,NaN,NaN,NaN,65.0,0.000000,-1.000000,504.00
1,OffHistorico,121120,CALDEIRA COMPACTA DE 30 T/H x 21 Kgf/Cmå_ x SA...,366.41,MONTAGEM,1.0,Caldeira,12.0,CC,1100.0,...,Operação/Projeto,1.470588,-154.865415,NaN,NaN,NaN,30.0,0.014706,-1.548654,366.41
2,OffHistorico,121140,CALDEIRA COMPACTA PARA QUEIMA DE G-S NATURAL 2...,992.25,MONTAGEM,1.0,Caldeira,12.0,CC,1100.0,...,Operação/Projeto,0.000000,-161.034560,NaN,NaN,NaN,25.0,0.000000,-1.610346,992.25
3,OffHistorico,121141,CALDEIRA AQUATUBULAR COMPCTA DE 65 t/h x 150 P...,2843.37,MONTAGEM,1.0,Caldeira,12.0,CC,1100.0,...,Operação/Projeto,11.764706,-111.681401,NaN,NaN,NaN,65.0,0.117647,-1.116814,2843.37
4,OffHistorico,121226,CALDEIRA AQUATUBULAR DE 65 T/H x 150 PSIG x SA...,1887.60,MONTAGEM,1.0,Caldeira,12.0,CC,1100.0,...,Operação/Projeto,11.764706,-111.681401,NaN,NaN,NaN,65.0,0.117647,-1.116814,1887.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,Fibria,Fibria,CRQ,19023.00,MONTAGEM,1.0,Caldeira,14.0,CRQ,1152.0,...,Economizador,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,19023.00
2434,Fibria,Fibria,CRQ,23736.00,SOLDA,1.0,Caldeira,14.0,CRQ,1132.0,...,Superaquecedor,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,23736.00
2435,Fibria,Fibria,CRQ,25853.00,CONFORMACAO,1.0,Caldeira,14.0,CRQ,1132.0,...,Superaquecedor,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,25853.00
2436,Fibria,Fibria,CRQ,34050.00,MONTAGEM,1.0,Caldeira,14.0,CRQ,1122.0,...,Fornalha,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,34050.00


In [4]:
offcod.to_excel('OffCod_Agru.xlsx', index=False)

In [5]:
# Ajusta a coluna de análise
filtrado2 = mais_frequentes["HorasUnit"]

#Transformação logarítmica
# filtrado2_log = np.log(filtrado2 + 1)

# Detecção de outliers
print("Detecção de outliers")
q1 = np.percentile(filtrado2, 25)
q3 = np.percentile(filtrado2, 75)
iqr = 0
lb, ub = q1 - 1.5 * iqr, q3 + 1.5 * iqr

# Converter limites de volta à escala original (considerando transfomação logarítmica)
# limite_inferior = np.exp(lb) - 1
# limite_superior = np.exp(ub) - 1

mais_frequentes["HorasUnit"] = filtrado2[(filtrado2 >= lb) & (filtrado2 <= ub)]


df_sem_outliers = mais_frequentes.dropna(subset="HorasUnit")

print(f"limite inferior: {lb} \n limite superior: {ub} \n Q1 = {q1} \n Q3 = {q3} \n IQR = {iqr}")

display(df_sem_outliers)

Detecção de outliers
limite inferior: 64.5 
 limite superior: 1474.0 
 Q1 = 64.5 
 Q3 = 1474.0 
 IQR = 0


,HorasUnit,Qtd,Poder.Z,Poder.Lin,Depart,Prod.Nome,Item.Cod
28,259.00,1.0,568.000000,100.000000,CONFORMACAO,CRQ,1111.0
33,539.00,1.0,-100.000000,12.000000,MONTAGEM,CC,1111.0
35,514.00,1.0,-65.000000,19.000000,MONTAGEM,CC,1111.0
37,1316.00,1.0,-208.000000,0.000000,MONTAGEM,CQCS,1111.0
39,1168.00,1.0,-96.000000,13.000000,MONTAGEM,CQOG,1111.0
...,...,...,...,...,...,...,...
1721,490.50,1.0,-40.189939,23.214286,SOLDA,CRQ,1921.0
1722,163.04,1.0,53.379421,41.071429,SOLDA,CRQ,1921.0
1723,774.94,1.0,6.594741,32.142857,SOLDA,CRQ,1921.0
1724,174.40,1.0,-115.045427,8.928571,SOLDA,CRQ,1921.0


#### Boxplot Duração X Tipo de Produto
----

In [6]:
pio.renderers.default = "plotly_mimetype"


fig = px.box(
    data_frame=my_data,
    x="TipodeProduto",  # Nome do produto
    y="SomaDeHorasApontadasUnitario",  # Duração
    color="TipodeProduto",  # Departamento (usado para agrupar por cor)
    title="Boxplot Duração X Tipo de Produto",  # Título do gráfico
    labels={"TipodeProduto": "Tipo de Produto", "SomaDeHorasApontadasUnitario": "Duração (Horas Unitárias)"},  # Rótulos personalizados
)

# Ajustando o layout
fig.update_layout(
    boxmode="group",  # Agrupamento dos boxplot    yaxis=dict(title="Duração"),  # Título do eixo Y
)

# Mostrando o gráfico
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

#### Boxplot Duração X Tipo de Produto (separado por departamento)
----

In [ ]:
fig = px.box(
    data_frame=my_data,
    x="TipodeProduto",  # Nome do produto
    y="SomaDeHorasApontadasUnitario",  # Duração
    color="Andrea_DEPARTAMENTO.Descricao",  # Departamento (usado para agrupar por cor)
    title="Boxplot Duração X Tipo de Produto (separado por departamento)",  # Título do gráfico
    labels={"TipodeProduto": "Tipo de Produto", "SomaDeHorasApontadasUnitario": "Duração (Horas Unitárias)", "Andrea_DEPARTAMENTO.Descricao": "Departamento" },  # Rótulos personalizados
)

# Ajustando o layout
fig.update_layout(
    boxmode="group",  # Agrupamento dos boxplots
    yaxis=dict(title="Duração"),  # Título do eixo Y
)

# Mostrando o gráfico
fig.show()

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1959: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1980: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



##### Nota-se que as caldeiras possuem uma maior frequência de duração em seu processo de fabricação

#### Boxplot Duração X Nome do Produto (itens mais frequentes e separados por departamento)
---

In [ ]:
fig = px.box(
    data_frame=mais_frequentes,
    x="Prod.Nome",  # Nome do produto
    y="HorasUnit",  # Duração
    color="Depart",  # Departamento (usado para agrupar por cor)
    title="Boxplot Duração X Produto (considerando apenas os itens mais frequentes)",  # Título do gráfico
    labels={"Prod.Nome": "Produto", "HorasUnit": "Duração (Horas Unitárias)"},  # Rótulos personalizados
)

# Ajustando o layout
fig.update_layout(
    boxmode="group",  # Agrupamento dos boxplots
    yaxis=dict(title="Duração"),  # Título do eixo Y
)

# Mostrando o gráfico
fig.show()

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1959: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1980: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



#### Boxplot Duração X Código do Item (considerando apenas os itens mais frequentes)
----

In [ ]:
# Criando o gráfico de boxplot
fig = px.box(
    data_frame=mais_frequentes,
    x="Item.Cod",  # Equivalente ao eixo x no R
    y="HorasUnit",  # Equivalente ao eixo y no R
    color="Depart",  # Agrupamento por cor
    title="Boxplot Duração X Código do Item",  # Título do gráfico
    labels={"Item.Cod": "Código dos Itens", "HorasUnit": "Duração (Horas Unitárias)", "Depart": "Departamento"}
)

# Ajustando o layout
fig.update_layout(
    boxmode="group",  # Agrupamento dos boxplots lado a lado
    yaxis=dict(title="Duração")  # Título do eixo Y
)

# Mostrando o gráfico
fig.show()

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1959: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1980: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



###### Obs: Foi observado que o item 1122 apresentou maior número de outliers no deparatamento traçagem, enquanto no item 1152, o maior número de outliers está presente no departamento solda. A solda possui maior número de outliers em relação aos outros departamentos.

#### Histograma Horas X Departamento
----

In [ ]:
# Criando o histograma com facetas
fig = px.histogram(
    my_data,
    x="SomaDeHorasApontadasUnitario",  # Eixo x com os valores da coluna
    color="Andrea_DEPARTAMENTO.Descricao",  # Cor baseada no departamento
    facet_col="Andrea_DEPARTAMENTO.Descricao",  # Facetando os gráficos com base no departamento
    title="Histograma de Frequência de Horas por Departamento",  # Título do gráfico
    labels={"SomaDeHorasApontadasUnitario": "Horas Unitárias", "Andrea_DEPARTAMENTO.Descricao": ""},  # Rótulos personalizados
    facet_col_wrap=3,  # Ajustando o número de facetas por linha
)

# Ajustando o layout para uma escala mais horizontal
fig.update_layout(
    width=800,  # Largura do gráfico
    height=600,  # Altura do gráfico
    #xaxis_title="Horas Unitárias",  # Título do eixo X
    #yaxis_title="Frequência",  # Título do eixo Y
)

# Mostrando o gráfico
fig.show()

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1959: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



#### Histograma Horas X Tipo de Produto
-----

In [ ]:
# Criando o histograma com facetas
fig = px.histogram(
    my_data,
    x="SomaDeHorasApontadasUnitario",  # Eixo x com os valores da coluna
    color="TipodeProduto",  # Cor baseada no departamento
    facet_col="TipodeProduto",  # Facetando os gráficos com base no departamento
    title="Histograma de Frequência de Horas por Tipo de Produto",  # Título do gráfico
    labels={"SomaDeHorasApontadasUnitario": "Horas Unitárias", "TipodeProduto": ""},  # Rótulos personalizados
    facet_col_wrap=3,  # Ajustando o número de facetas por linha
)

# Ajustando o layout para uma escala mais horizontal
fig.update_layout(
    width=800,  # Largura do gráfico
    height=600,  # Altura do gráfico
    #xaxis_title="Horas Unitárias",  # Título do eixo X
    #yaxis_title="Frequência",  # Título do eixo Y
)

# Mostrando o gráfico
fig.show()

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1959: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



##### Observa-se que a caldeira é o produto com a maior frequência de horas 

#### Histograma Duração X Itens mais frequentes
----

In [ ]:
# frequentes_departamento = mais_frequentes[mais_frequentes['Depart']]

# Criando o histograma
fig = px.histogram(
    data_frame=mais_frequentes,
    x="HorasUnit",  # Variável no eixo x
    color="Item.Cod",  # Preenchimento baseado no código do item
    facet_col="Item.Cod",  # Criando painéis separados para cada Item.Cod
    facet_col_wrap=8,  # Ajustando o número de facetas por linha
    title="Histograma Duração X Código do Item (entre os mais frequentes)",
    labels={"HorasUnit": "", "Item.Cod":""},  # Rótulos personalizados
    opacity=0.75  # Ajuste de transparência
)

# Ajustando o layout
fig.update_layout(
    bargap=0.1,  # Espaço entre as barras
    showlegend=False,  # Ocultar a legenda (opcional)
)

# Mostrando o gráfico
fig.show()

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1959: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



#### Gráfico de Dispersão Soma de Horas X Poder Linear
----

In [ ]:
my_data_PL_notNA = my_data[my_data['Poder Linear'].notna()]

# Criando o gráfico de dispersão
fig = px.scatter(
    data_frame=my_data_PL_notNA,  # DataFrame contendo os dados
    x="SomaDeHorasApontadasUnitario",  # Coluna no eixo X
    y="Poder Linear",  # Coluna no eixo Y
    symbol="Andrea_DEPARTAMENTO.Descricao",  # Coluna para diferenciar os pontos por símbolo
    color= "Andrea_DEPARTAMENTO.Descricao",
    title="Relação entre Soma de Horas Apontadas Unitário e Poder Linear",  # Título do gráfico
    labels={
        "SomaDeHorasApontadasUnitario": "Soma de Horas Apontadas Unitário",
        "Poder Linear": "Poder Linear",
        "Andrea_DEPARTAMENTO.Descricao": "Departamento"
    },  # Rótulos personalizados
)

# Ajustando o layout
fig.update_layout(
    xaxis=dict(title="Soma de Horas Apontadas Unitário"),
    yaxis=dict(title="Poder Linear"),
    legend=dict(title="Departamento"),
    modebar_add=["resetScale2d", "hoverClosestCartesian"],
)

# Mostrando o gráfico
fig.show()

c:\Users\Pedro Amaral Pereira\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:1959: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



#### Gráfico de Densidade por tipo de Produto
---- 

In [ ]:
# Tipos de produto e seus dados correspondentes
produtos = {
    "Caldeira": my_data.loc[my_data['TipodeProduto'] == 'Caldeira', 'SomaDeHorasApontadasUnitario'],
    "Trocador de Calor": my_data.loc[my_data['TipodeProduto'] == 'Trocador de Calor', 'SomaDeHorasApontadasUnitario'],
    "Vasos": my_data.loc[my_data['TipodeProduto'] == 'Vasos', 'SomaDeHorasApontadasUnitario'],
    "Cilindro de Cloro": my_data.loc[my_data['TipodeProduto'] == 'Cilindro de Cloro', 'SomaDeHorasApontadasUnitario'],
    "Turbina": my_data.loc[my_data['TipodeProduto'] == 'Turbina', 'SomaDeHorasApontadasUnitario'],
    "Precipitador": my_data.loc[my_data['TipodeProduto'] == 'Precipitador', 'SomaDeHorasApontadasUnitario'],
    "Outros": my_data.loc[my_data['TipodeProduto'] == 'Outros', 'SomaDeHorasApontadasUnitario']
}

# Determinando o intervalo comum para o eixo X
x_vals = np.linspace(
    min(data.min() for data in produtos.values() if not data.empty),
    max(data.max() for data in produtos.values() if not data.empty),
    500
)

# Criando o gráfico
fig = go.Figure()

# Calculando densidades e adicionando traços ao gráfico
for nome, dados in produtos.items():
    if not dados.empty:
        densidade = gaussian_kde(dados.dropna())  # Calcula densidade
        fig.add_trace(go.Scatter(
            x=x_vals, y=densidade(x_vals),
            mode='lines', fill='tozeroy',
            name=nome
        ))

# Ajustando o layout
fig.update_layout(
    title="Densidade X Horas Apontadas por tipo de produto",
    xaxis=dict(title='Horas apontadas'),
    yaxis=dict(title='Densidade'),
    legend_title="Produto"
)

# Mostrando o gráfico
fig.show()